In [3]:
import numpy as np
import pandas as pd

np.random.seed(1337)  # for reproducibility
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.regression import r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler

from dbn.tensorflow import SupervisedDBNRegression

In [4]:
# Read the dataset
ROAD = "Taft Ave."
YEAR = "2015"
EXT = ".csv"
FILENAME = "merged_mmda_wwo_" + ROAD + "_" + YEAR
original_dataset = pd.read_csv("data/mmda-wwo/" + FILENAME + EXT, skipinitialspace=True)

##### Preparing Traffic Dataset

In [5]:
original_dataset = pd.read_csv("data/mmda-wwo/" + FILENAME + EXT, skipinitialspace=True)
traffic_dataset = original_dataset
print("Start : " + str(original_dataset.columns[0:original_dataset.shape[1]][5]))
print("End : " + str(original_dataset.columns[0:original_dataset.shape[1]][traffic_dataset.shape[1]-1]))

# Remove date time. Remove unused columms
#0-2 = dt + lineName + stationName || 3-4 - statusN - statusS || 5-end - weather variables
cols_to_remove = [0, 1, 2] + list(range(5, traffic_dataset.shape[1]))

# #Remove Redundant Variables
# #Variables = WindspeedKmph, cond, precipMM, humidity, visibility, pressure, cloudcover, dewPointC, windGustKmph, 
# redundant_variables = [5, 6, 7, 9, 16, 17, 19, 20, 21, 22, 24, 25]
# cols_to_remove += redundant_variables

traffic_dataset.drop(traffic_dataset.columns[[cols_to_remove]], axis=1, inplace=True)
traffic_dataset.head()

Start : tempC
End : feelsLikeF


,statusN,statusS
0,0.5,0.5
1,0.5,0.5
2,0.5,0.5
3,0.5,0.5
4,0.5,0.5


##### Preparing Weather Dataset

In [6]:
original_dataset = pd.read_csv("data/mmda-wwo/" + FILENAME + EXT, skipinitialspace=True)
weather_dataset = original_dataset
print("Start : " + str(weather_dataset.columns[0:original_dataset.shape[1]][5]))
print("End : " + str(weather_dataset.columns[0:original_dataset.shape[1]][weather_dataset.shape[1]-1]))

# Remove date time. Remove unused columms
#0-2 = dt + lineName + stationName || 3-4 - statusN - statusS || 5-end - weather variables
cols_to_remove = [0, 1, 2] + [3, 4]

#Remove Redundant Variables
#Variables = tempC WindspeedKmph, cond, precipMM, humidity, visibility, pressure, cloudcover, dewPointC, windGustKmph, 
redundant_variables = [6, 7, 9, 16, 17, 19, 20, 21, 22, 24, 25]
cols_to_remove += redundant_variables

weather_dataset.drop(weather_dataset.columns[[cols_to_remove]], axis=1, inplace=True)
weather_dataset.head()

Start : tempC
End : feelsLikeF


,tempC,windspeedKmph,cond,precipMM,humidity,visibility,pressure,cloudcover,dewPointC,windGustKmph
0,0.2000,0.295455,0.631579,0.0,0.84058,1.0,0.758621,0.330,0.538462,0.236111
1,0.1875,0.295455,0.631579,0.0,0.84058,1.0,0.758621,0.325,0.538462,0.239583
2,0.1750,0.295455,0.631579,0.0,0.84058,1.0,0.758621,0.320,0.538462,0.243056
3,0.1625,0.295455,0.631579,0.0,0.84058,1.0,0.758621,0.315,0.538462,0.246528
4,0.1500,0.295455,0.631579,0.0,0.84058,1.0,0.758621,0.310,0.538462,0.250000


In [7]:
# To-be Predicted variable
Y = traffic_dataset.statusS
Y = Y.round(5)

# Other data
X = weather_dataset
#X = dataset
#X.statusS = X.statusS.round(5)

# Splitting data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.67, shuffle=False)
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

# Data scaling
min_max_scaler = MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)

In [8]:
# Training
regressor = SupervisedDBNRegression(hidden_layers_structure=[5, 10, 15],
                                    learning_rate_rbm=0.01,
                                    learning_rate=0.01,
                                    n_epochs_rbm=3,
                                    n_iter_backprop=5,
                                    batch_size=16,
                                    activation_function='relu')
regressor.fit(X_train, Y_train)


[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 0.395865
>> Epoch 2 finished 	RBM Reconstruction error 0.300670
>> Epoch 3 finished 	RBM Reconstruction error 0.227536
>> Epoch 1 finished 	RBM Reconstruction error 0.114518
>> Epoch 2 finished 	RBM Reconstruction error 0.076322
>> Epoch 3 finished 	RBM Reconstruction error 0.048765
>> Epoch 1 finished 	RBM Reconstruction error 0.055207
>> Epoch 2 finished 	RBM Reconstruction error 0.049728
>> Epoch 3 finished 	RBM Reconstruction error 0.040936
[END] Pre-training step
[START] Fine tuning step:
>> Epoch 0 finished 	ANN training loss 0.117848
>> Epoch 1 finished 	ANN training loss 0.113627
>> Epoch 2 finished 	ANN training loss 0.110349
>> Epoch 3 finished 	ANN training loss 0.108246
>> Epoch 4 finished 	ANN training loss 0.106095
[END] Fine tuning step


SupervisedDBNRegression(batch_size=16, dropout_p=0, l2_regularization=1.0,
            learning_rate=0.01, n_iter_backprop=5, verbose=True)

In [9]:
# Test
X_test = min_max_scaler.transform(X_test)
Y_pred = regressor.predict(X_test)
print('Done.\nR-squared: %f\nMSE: %f' % (r2_score(Y_test, Y_pred), mean_squared_error(Y_test, Y_pred)))


Done.
R-squared: -0.023860
MSE: 0.120031


In [10]:
print(len(Y_pred))
temp = []
for i in range(len(Y_pred)):
    temp.append(Y_pred[i][0])
d = {'Predicted': temp, 'Actual': Y_test}

df = pd.DataFrame(data=d)
df.head()

23477


,Actual,Predicted
0,1.0,0.886714
1,1.0,0.890329
2,1.0,0.894399
3,1.0,0.898469
4,1.0,0.902540


In [11]:
df

,Actual,Predicted
0,1.0,0.886714
1,1.0,0.890329
2,1.0,0.894399
3,1.0,0.898469
4,1.0,0.902540
5,0.5,0.906610
6,0.5,0.907247
7,0.5,0.907885
8,0.5,0.908522
9,0.5,0.909160


In [12]:
# Save the model
regressor.save('models/pm2-noeng.pkl')

# # Restore
# classifier = SupervisedDBNClassification.load('model.pkl')

# # Test
# Y_pred = classifier.predict(X_test)
# print('Done.\nAccuracy: %f' % accuracy_score(Y_test, Y_pred))


In [13]:
df.to_csv("output/pm2_noeng_output_" + FILENAME + EXT, encoding='utf-8', index=False)